In [1]:
import pandas as pd
import matplotlib.pyplot as plt
# from matplotlib import rc
import seaborn as sns
import os
import glob
sns.set(style='ticks')
import json
s = json.load( open(r"../custom.json"))  #edit path to json file\n",
plt.rcParams.update(s)

In [2]:
# file =r"E:\Documents\Research\Molecular Dynamics\Code\GitHub\Jupyter\Cross Coeff Mie Pot\coeff_nn.csv"
names = []

In [3]:
# atoms = ["C","CH3","CH2"]
atoms = ["CH3","CH2","C","CO2","Wall"]
atom_index = [1,2,3,4,5]
ep = [0.175, 0.118,0.070,0.4475,1.0]
sig = [3.905,3.905,3.55, 3.7900,1.0]
# df = pd.DataFrame(list(zip(lst, lst2)), 
#                columns =['Name', 'val']) 
df = pd.DataFrame(list(zip(atoms,atom_index, ep, sig)),columns=["atom","type","ep (kcal/mol)","sigma (A)"])
# df["pair_coeff"]="pair_coeff"
# df.set_index("pair_coeff",inplace=True)

In [4]:
df

,atom,type,ep (kcal/mol),sigma (A)
0,CH3,1,0.1750,3.905
1,CH2,2,0.1180,3.905
2,C,3,0.0700,3.550
3,CO2,4,0.4475,3.790
4,Wall,5,1.0000,1.000


In [5]:
def cal(sig_ii,sig_jj,ep_ii, ep_jj):
    sig_ij = round((sig_ii+sig_jj)/2,4);
    ep_ij = round((ep_ii*ep_jj)**0.5,4)
    return (sig_ij,ep_ij)
 

In [6]:
# pc_df = pd.DataFrame(columns=["i","j","sig_ij"])
print()
# print(mol_list["Methane"].index())
pc_list =[]
for mol_i in df["atom"]:
    index_mol_i = df.index[df.atom ==mol_i][0]
    for mol_j in df.iloc[index_mol_i:]["atom"]:
        loc_i = df.loc[df["atom"]==mol_i]
        loc_j = df.loc[df["atom"]==mol_j]
        sig_ii = loc_i["sigma (A)"].values[0]
        sig_jj = loc_j["sigma (A)"].values[0]
        ep_ii = loc_i["ep (kcal/mol)"].values[0]
        ep_jj = loc_j["ep (kcal/mol)"].values[0]
        
        cross_coeff = cal(sig_ii,sig_jj,ep_ii, ep_jj)
        pc_list.append({"i":loc_i["type"].values[0],"j":loc_j["type"].values[0],"sig_ij":cross_coeff[0],
                       "ep_ij":cross_coeff[1],"interaction":f"#{loc_i['atom'].values[0]}-{loc_j['atom'].values[0]}"
                       })
#         print({"i":[loc_i["Type"].values[0]],"j":[loc_j["Type"].values[0]],"sig_ij":[sig_ij(sig_ii,sig_jj)]})
pc_df =pd.DataFrame(pc_list, columns=["pair_coeff","i","j","ep_ij","sig_ij","interaction"])
pc_df["pair_coeff"]="pair_coeff"
pc_df["cut_off"]=13.0
# pc_df["pot"] ="lj/cut"
# pc_df.loc[(pc_df["i"]==11)|(pc_df["j"]==11),"pot"]="lj/cut"
# pc_df.loc[(pc_df["pot"]=="lj/cut"),["lmd_ij","lmd_a"]]=""
# pc_df.set_index("pair_coeff",inplace=True)

pc_df.sort_values(by=["i","j"],inplace=True)
pc_df = pc_df[["pair_coeff","i","j","ep_ij","sig_ij","cut_off","interaction"]]
# pc_df.loc[(pc_df['j'] ==5)]['ep_ij','sig_jj'] =1.0

pc_df

,pair_coeff,i,j,ep_ij,sig_ij,cut_off,interaction
0,pair_coeff,1,1,0.1750,3.9050,13.0,#CH3-CH3
1,pair_coeff,1,2,0.1437,3.9050,13.0,#CH3-CH2
2,pair_coeff,1,3,0.1107,3.7275,13.0,#CH3-C
3,pair_coeff,1,4,0.2798,3.8475,13.0,#CH3-CO2
4,pair_coeff,1,5,0.4183,2.4525,13.0,#CH3-Wall
5,pair_coeff,2,2,0.1180,3.9050,13.0,#CH2-CH2
6,pair_coeff,2,3,0.0909,3.7275,13.0,#CH2-C
7,pair_coeff,2,4,0.2298,3.8475,13.0,#CH2-CO2
8,pair_coeff,2,5,0.3435,2.4525,13.0,#CH2-Wall
9,pair_coeff,3,3,0.0700,3.5500,13.0,#C-C


In [7]:
pc_df.describe()

,i,j,ep_ij,sig_ij,cut_off
count,15.000000,15.000000,15.000000,15.000000,15.0
mean,2.333333,3.666667,0.302520,3.230000,13.0
std,1.290994,1.290994,0.252284,0.887423,0.0
min,1.000000,1.000000,0.070000,1.000000,13.0
25%,1.000000,3.000000,0.130850,2.452500,13.0
50%,2.000000,4.000000,0.229800,3.727500,13.0
75%,3.000000,5.000000,0.380900,3.847500,13.0
max,5.000000,5.000000,1.000000,3.905000,13.0


In [8]:
pc_df.loc[(pc_df['i'] == 5.0) | (pc_df['j'] == 5.0),'ep_ij'] = 1.0
pc_df.loc[(pc_df['i'] == 5.0) | (pc_df['j'] == 5.0),'sig_ij'] = 2.0
pc_df.loc[(pc_df['i'] == 5.0) | (pc_df['j'] == 5.0),['cut_off']] = round(2.0*2**(1/6),4)
pc_df.loc[(pc_df['i'] == 3.0) & (pc_df['j'] == 5.0),['cut_off','ep_ij','sig_ij']] = 0
pc_df.loc[(pc_df['i'] == 5.0) & (pc_df['j'] == 5.0),['cut_off','ep_ij','sig_ij']] = 0
pc_df.loc[(pc_df['i'] == 3.0) & (pc_df['j'] == 3.0),['cut_off','ep_ij','sig_ij']] = 0
pc_df.set_index("pair_coeff",inplace=True)
pc_df

,i,j,ep_ij,sig_ij,cut_off,interaction
pair_coeff,,,,,,
pair_coeff,1,1,0.1750,3.9050,13.0000,#CH3-CH3
pair_coeff,1,2,0.1437,3.9050,13.0000,#CH3-CH2
pair_coeff,1,3,0.1107,3.7275,13.0000,#CH3-C
pair_coeff,1,4,0.2798,3.8475,13.0000,#CH3-CO2
pair_coeff,1,5,1.0000,2.0000,2.2449,#CH3-Wall
pair_coeff,2,2,0.1180,3.9050,13.0000,#CH2-CH2
pair_coeff,2,3,0.0909,3.7275,13.0000,#CH2-C
pair_coeff,2,4,0.2298,3.8475,13.0000,#CH2-CO2
pair_coeff,2,5,1.0000,2.0000,2.2449,#CH2-Wall


In [9]:
pc_df.to_csv("pot.txt",sep=" ")
# 

In [10]:
df.loc[(df['A'] == 'blue') & (df['B'] == 'red') & (df['C'] == 'square'),'D'] = 'M5

SyntaxError: EOL while scanning string literal (<ipython-input-10-ea44509bcdf6>, line 1)